In [1]:
import sys
sys.path.insert(0, r'../')
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.optim as optim
import torch.nn as nn
import torch
import pickle
from sklearn.model_selection import train_test_split
import os
from utils.preprocess import pad_sequences
from utils.F1 import F1
from net.FS import FS
from net.FE import FE
from net.net2 import CNN_GAP2
from dataset.mydataset import myDataset
from utils.freeze_model import freeze, unfreeze
from utils.f1_loss import f1_loss
from utils.wce import wce

In [2]:
a = torch.tensor([1, 3, 2, 0, 3, 2])
a

tensor([1, 3, 2, 0, 3, 2])

In [3]:
# one-hot 
b = torch.zeros(a.size()[0],4)
b[torch.arange(a.size()[0]),a] = 1
b

tensor([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]])

In [4]:
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
pred = torch.rand((10,4))
true = torch.tensor([2,0,3,1,1,1,1,1,1,1])
print("PRED", pred)
print("TRUE", true)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1,1/7,1,1]).float())
print(criterion(pred,true))
print(wce(pred, true,  n_class=4))

PRED tensor([[0.9440, 0.5546, 0.9885, 0.2473],
        [0.0790, 0.7572, 0.9206, 0.7698],
        [0.4257, 0.8066, 0.6271, 0.4482],
        [0.8774, 0.9506, 0.1028, 0.5658],
        [0.3213, 0.8262, 0.6814, 0.6837],
        [0.5272, 0.3485, 0.5280, 0.9197],
        [0.5998, 0.1928, 0.3436, 0.9958],
        [0.3696, 0.6641, 0.0472, 0.4701],
        [0.0950, 0.7347, 0.5834, 0.2356],
        [0.0021, 0.4682, 0.4577, 0.7574]])
TRUE tensor([2, 0, 3, 1, 1, 1, 1, 1, 1, 1])
tensor(1.4930)
tensor(1.4930)


In [33]:
pred.requires_grad = True
pred.grad.data.zero_()
print(pred.grad)
loss = wce(pred, true,  n_class=4)
pred.retain_grad()
loss.backward()
print(pred.grad)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.0776,  0.0526, -0.1689,  0.0387],
        [-0.2157,  0.0676,  0.0796,  0.0685],
        [ 0.0531,  0.0777,  0.0649, -0.1957],
        [ 0.0109, -0.0240,  0.0050,  0.0080],
        [ 0.0065, -0.0250,  0.0093,  0.0093],
        [ 0.0083, -0.0288,  0.0083,  0.0122],
        [ 0.0091, -0.0297,  0.0070,  0.0135],
        [ 0.0086, -0.0242,  0.0062,  0.0095],
        [ 0.0063, -0.0238,  0.0103,  0.0072],
        [ 0.0057, -0.0267,  0.0089,  0.0121]])


In [31]:
pred.requires_grad = True
pred.grad.data.zero_()
print(pred.grad)
loss = criterion(pred, true)
pred.retain_grad()
loss.backward()
print(pred.grad)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.0776,  0.0526, -0.1689,  0.0387],
        [-0.2157,  0.0676,  0.0796,  0.0685],
        [ 0.0531,  0.0777,  0.0649, -0.1957],
        [ 0.0109, -0.0240,  0.0050,  0.0080],
        [ 0.0065, -0.0250,  0.0093,  0.0093],
        [ 0.0083, -0.0288,  0.0083,  0.0122],
        [ 0.0091, -0.0297,  0.0070,  0.0135],
        [ 0.0086, -0.0242,  0.0062,  0.0095],
        [ 0.0063, -0.0238,  0.0103,  0.0072],
        [ 0.0057, -0.0267,  0.0089,  0.0121]])
